In [1]:
from denoising_diffusion_pytorch import Unet, GaussianDiffusion, Sampler, DPS, DenoiseOperator, AnisotropicOperator

import torch
import numpy as np
from torch.cuda.amp import autocast as autocast
from torchvision import utils

/home/ryuuyou/.conda/envs/dm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Data

In [2]:
data_path = "/home/share/CARE/Isotropic_Liver/train_data/data_label.npz"
data_file = np.load(data_path)
lr_data = data_file['X']

In [3]:
one_data = torch.from_numpy(lr_data[0:1]).cuda()
print(one_data.shape)

torch.Size([1, 1, 128, 128])


# model HR

In [4]:
model_hr = Unet(
    dim = 64,
    channels=1,
    dim_mults = (1, 2, 4, 8)
)
diffusion_hr = DPS(
    model_hr,
    image_size = 128,
    timesteps = 1000,
    sampling_timesteps = 250,
)
sampler = Sampler(diffusion_hr)
hr_checkpoint_path = "/home/share_ssd/ryuuyou/denoising-diffusion/pretrained_y/model_150k_steps_lr1e-5.pt"
sampler.load(path=hr_checkpoint_path)

loading from: [version]:1.5.4; [step]:150000


# Operator

In [5]:
operator = AnisotropicOperator(img_shape=one_data.shape)
# operator = DenoiseOperator()

# Inference

In [6]:
return_all_timesteps = True
with autocast():
    res = sampler.dps(measurement=one_data, operator=operator, num_samples=16, return_all_timesteps=return_all_timesteps)

sampling loop time step: 100%|██████████| 250/250 [00:32<00:00,  7.78it/s]


In [7]:
print(res.shape)

torch.Size([251, 16, 1, 128, 128])


In [8]:
sampler.save(one_data, path='./out/input.png')
if not return_all_timesteps:
    sampler.save(res, path='./out/res.png')
else:
    sampler.save_with_records(res, folder='./out/record/')